<a id="learn2-top"></a> 
# Bloomberg Python Series: <br><span style="color:orange"> Using the Bloomberg Query Language with Fixed Income Data </span>

### Topics

1. [__Key Benefits__](#learn2-topic1)
2. [__Basic Features__](#learn2-topic2)
- [2.1 How To Start](#learn2-topic2.1)
- [2.2 Field Rationalisation](#learn2-topic2.2)
- [2.3 Custom calculation](#learn2-topic2.3)
- [2.4 Data Availability Management](#learn2-topic2.4)
3. [__Advanced Features__](#learn2-topic3)
- [3.1 Chains](#learn2-topic3.1)
- [3.2 Output Manipulation in BQL](#learn2-topic3.2)
- [3.3 Filtering/Screening](#learn2-topic3.3)
    - [3.3.1 Filter by Single Condition](#learn2-topic3.3.1)
    - [3.3.2 Filter by Multiple Condition](#learn2-topic3.3.2)
- [3.4 Index Members](#learn2-topic3.4)
- [3.5 Grouping](#learn2-topic3.5)
- [3.6 Portfolio Access](#learn2-topic3.6)
- [3.7 Universes](#learn2-topic3.7)



<a id='learn2-topic1'></a>
<span style="color:darkorange; font-size:2em"> 1 Key Benefits </span>



- Faster, more efficient execution in Bloomberg Cloud

<img src="../../Visualisations/BQL Process.jpg" style="width: 600px;"/>


- Clean, standardized data for cross asset analysis
- Instead invest time and resources into generating ideas and alpha

[&uarr; Return to Top](#learn2-top)

<a id='learn2-topic2'></a>
<span style="color:darkorange; font-size:2em"> 2 Basic features </span>

<a id='learn2-topic2.1'></a>
### 2.1 How to Start



Import package for BQL

In [1]:
import bql

Then connect to the BQL Service to Databases by bql.Service()

In [2]:
# always have this in one cell and keep it on the top, because "bq" can be reused after
bq = bql.Service()

<a id='learn2-topic2.2'></a>
### 2.2 Field Rationalization



BQL standardizes the fields required for data retrieval to make it easier and more efficient to retrieve data. eg

<img src="../../Visualisations/BDP vs BQL.jpg" style="width: 600px;"/>

See the BQL Editor for field explanations, parameters and defaults

In [3]:
# Getting OAS spread for a bond
# Universe is our bond
bond = 'ZS7759443 Corp'
# Define our OAS spread
spread = bq.data.SPREAD(SPREAD_TYPE='OAS')
#Shift + Tab after a data item shows the available parameters 

req = bql.Request(bond, spread)
res = bq.execute(req)
data = res[0].df()
data

,DATE,SPREAD(spread_type=SPREAD_TYPE.OAS)
ID,,
ZS7759443 Corp,2021-04-08,50.001827


<a id='learn2-topic2.3'></a>

### 2.3 Custom Calculation:



- How to retrieve dates in BQL
- request multiple fields and perform operations

Absolute dates, always in YYYY-MM-DD

In [4]:
#Time-series of price

#Select our bond
bond = 'ZS7759443 Corp'
# Absolute dates, always in YYYY-MM-DD
px = bq.data.PX_LAST(dates=bq.func.range('2019-12-07','2019-12-12'))
# Good practice to create a DateTime object

req = bql.Request(bond, px)
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,"PX_LAST(dates=RANGE(2019-12-07,2019-12-12))"
ID,,,
ZS7759443 Corp,2019-12-07,None,NaN
ZS7759443 Corp,2019-12-08,None,NaN
ZS7759443 Corp,2019-12-09,None,105.923
ZS7759443 Corp,2019-12-10,None,105.908
ZS7759443 Corp,2019-12-11,None,106.206
ZS7759443 Corp,2019-12-12,None,105.869


Relative dates can be expressed in days, weeks, months, quarters, semi-annual, years

In [5]:
#Select our bond
bond = 'ZS7759443 Corp'
#Relative dates can be expressed in days, weeks, months, quarters, semi-annual, years
px_1w = bq.data.PX_LAST(dates=bq.func.range('-1w','0d'), PRICING_SOURCE='BVAL')

req = bql.Request(bond, {'Price over 1 week':px_1w})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Price over 1 week
ID,,,
ZS7759443 Corp,2021-04-01,None,110.419098
ZS7759443 Corp,2021-04-02,None,NaN
ZS7759443 Corp,2021-04-03,None,NaN
ZS7759443 Corp,2021-04-04,None,NaN
ZS7759443 Corp,2021-04-05,None,NaN
ZS7759443 Corp,2021-04-06,None,110.586403
ZS7759443 Corp,2021-04-07,None,110.754204
ZS7759443 Corp,2021-04-08,None,110.912735


#### 2.3.1 Custom Calculation Practice

What was the 20d net change of BVAL YTW Yield for QZ095290 Corp?

In [6]:
# Type code here
bond = 'QZ095290 Corp'
yld_chg = bq.data.yield_(dates=bq.func.range('-20d','0d'), PRICING_SOURCE='BVAL', fill='prev').pct_chg()

req = bql.Request(bond, {'Yield Change':yld_chg})
res = bq.execute(req)
data = res[0].df()
data

,DATE,Yield Change
ID,,
QZ095290 Corp,2021-04-08,-41.127037


<a id='learn2-topic2.4'></a>

### 2.4 Data availability management



- managing n/a in a time series
- setting a rule for n/a's (dropping and zeroes)

In [7]:
#Getting yield for the last week
bond = 'ZS7759443 Corp'
#A parameter or a function can manage the unavailable data
yld = bq.data.PX_LAST(dates=bq.func.range('-1w','0d'), PRICING_SOURCE='BVAL').dropna()

req = bql.Request(bond, yld)
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,"DROPNA(PX_LAST(dates=RANGE(-1W,0D),pricing_source=PRICING_SOURCE.BVAL))"
ID,,,
ZS7759443 Corp,2021-04-01,None,110.419098
ZS7759443 Corp,2021-04-06,None,110.586403
ZS7759443 Corp,2021-04-07,None,110.754204
ZS7759443 Corp,2021-04-08,None,110.912735


Unique fields like 'CHG_PCT_5D' (Percent change 5 day) have all been replaced by BQL Functions that apply to a BQL field.
This allows for flexible changes in calculations and parameters.


In [8]:
#Percentage Change of Z-Spread for 5 day history

bond = 'ZS7759443 Corp'
sprd_chg = bq.data.SPREAD(dates=bq.func.range('-5d','0d'), fill='prev').pct_chg()

req = bql.Request(bond, sprd_chg)
res = bq.execute(req)
data = res[0].df()
data

,DATE,"PCT_CHG(SPREAD(fill=FILL.prev,dates=RANGE(-5D,0D)))"
ID,,
ZS7759443 Corp,2021-04-08,-6.777765


[&uarr; Return to Top](#learn2-top)

<a id='learn2-topic3'></a>

<span style="color:darkorange; font-size:2em"> 3 Advanced Features </span>



<a id='learn2-topic3.1'></a>
### 3.1 Chains:



- Retrieving bonds, loans, debt data in BQL
- filtering data in BQL

In [10]:
#Pulling all of Daimler's bonds
bonds = bq.univ.bonds('DAI GR Equity')
security = bq.data.name()

req = bql.Request(bonds, security)
res = bq.execute(req)
data = res[0].df().head()
data

,NAME()
ID,
BO398060 Corp,DAIGR 0 ¾ 03/11/33
ZO315619 Corp,DAIGR 0 ¾ 09/10/30
AO113936 Corp,DAIGR 2 ⅛ 07/03/37
AZ907030 Corp,DAIGR 0 ⅜ 11/08/26
ZQ376412 Corp,DAIGR 0 ⅝ 05/06/27


Within BQL, we can specify the corporate structure from which to pull securities

- Entity: 	    Securities by specified entity only
- Entity_Subs:	Downloads securities issued by specified entity and direct subsidiaries
- Credit Family:	Downloads securities issued by the parent company and all subsidiaries

<img src="../../Visualisations/Issuedby.jpg" style="width: 600px;"/>

In [11]:
#Pulling all of Rio Tinto's bonds issued under their credit family:
bonds = bq.univ.bonds('RIO AU Equity', issuedby='Credit_Family')
security = bq.data.ID()

req = bql.Request(bonds, security)
res = bq.execute(req)
data = res[0].df().head()
data

,ORIG_IDS,ID()
ID,,
EK974172 Corp,RIO AU Equity,EK974172 Corp
EI452667 Corp,RIO AU Equity,EI452667 Corp
EH437851 Corp,RIO AU Equity,EH437851 Corp


We can apply this for multiple issuers and larger universes.

In [12]:
#get the bonds for IBM and Apple, paying extra attention to the brackets for a list
bonds = bq.univ.bonds(['IBM US Equity','AAPL US Equity'])
securityid = bq.data.ID()

req = bql.Request(bonds, securityid)
res = bq.execute(req)
data = res[0].df().head(10)
data

,ORIG_IDS,ID()
ID,,
DD103619 Corp,IBM US Equity,DD103619 Corp
ZS542665 Corp,IBM US Equity,ZS542665 Corp
BJ226369 Corp,IBM US Equity,BJ226369 Corp
BJ226366 Corp,IBM US Equity,BJ226366 Corp
ZS542668 Corp,IBM US Equity,ZS542668 Corp
ZS542663 Corp,IBM US Equity,ZS542663 Corp
BJ226365 Corp,IBM US Equity,BJ226365 Corp
DD103620 Corp,IBM US Equity,DD103620 Corp
ZS542664 Corp,IBM US Equity,ZS542664 Corp


<a id='learn2-topic3.2'></a>
### 3.2 Output Manipulation in BQL:



- Manipulate string
- Manipulate values 

In [13]:
from collections import OrderedDict
import pandas as pd

In [14]:
#How do I see the year and month of maturity and yield for all of Virgin Media Secured Finance (0533805D LN) bonds and loans, rounded by 3 decimal points?

#define our debt
debt = bq.univ.debt('0533805D LN Equity', issuedby='Entity')
#Maturity
mat = bq.data.maturity()
mat_year = mat.year()
mat_month = mat.month()
mat_date = mat_year * 100 + mat_month
#Yield
yld = bq.data.yield_()
rounded_yld = yld.round('3')
#Put into our dictionary
flds = OrderedDict()
flds['Maturity'] = mat_date
flds['Yield'] = rounded_yld

req = bql.Request(debt, flds)
res = bq.execute(req)

tbl = pd.DataFrame({r.name:r.df()[r.name] for r in res})
tbl.head(20)

,Maturity,Yield
ID,,
ZR870724 Corp,203001.0,4.011
BK131082 Corp,203008.0,4.162
ZS539065 Corp,202905.0,3.805
BK120206 Corp,203008.0,4.000
AM204429 Corp,202704.0,2.520
JK774647 Corp,202608.0,2.044
ZS539067 Corp,202905.0,3.561
AM445380 Corp,202501.0,NaN


<a id='learn2-topic3.3'></a>
### 3.3 Filtering / Screening



BQL allows us to screen our universe directly on the Bloomberg Server. We can access this with bq.univ.filter()

<img src="../../Visualisations/BQL Filtering.jpg" style="width: 600px;"/>

<a id='learn2-topic3.3.1'></a>

### 3.3.1 Filter by Single Condition



- filter(Universe, Condition)

In [15]:
#Get only the fixed coupon bonds for Vodafone
bonds = bq.univ.bonds(['VOD LN Equity'])
filter_cpn = bq.data.CPN_TYP() == 'FIXED'
screen = bq.univ.filter(bonds, filter_cpn)
#Define our output
securityid = bq.data.ID()

req = bql.Request(screen, securityid)
res = bq.execute(req)

#let's just see the first 20
data = res[0].df().head(20)
data

,ORIG_IDS,ID()
ID,,
AS779476 Corp,VOD LN Equity,AS779476 Corp
EJ552316 Corp,VOD LN Equity,EJ552316 Corp
JK203775 Corp,VOD LN Equity,JK203775 Corp
EJ372541 Corp,VOD LN Equity,EJ372541 Corp
QZ698193 Corp,VOD LN Equity,QZ698193 Corp
EI109926 Corp,VOD LN Equity,EI109926 Corp
AM660337 Corp,VOD LN Equity,AM660337 Corp
ZR549306 Corp,VOD LN Equity,ZR549306 Corp
JK189339 Corp,VOD LN Equity,JK189339 Corp


<a id='learn2-topic3.3.2'></a>
### 3.3.2 Filter by Multiple Conditions



- filter(Universe, Condition1 AND Condition2 OR Condition3)

In [16]:
#apply multiple filter criteria within our request, eg get fixed coupon bonds for Vodafone with a coupon above 1
bonds = bq.univ.bonds(['VOD LN Equity'])
filter_cpntyp = bq.data.CPN_TYP() == 'FIXED'
filter_cpn = bq.data.CPN() > 1
criteria = bq.func.and_(filter_cpntyp, filter_cpn)
screen = bq.univ.filter(bonds, criteria)
#Get the security IDs
securityid = bq.data.ID()

req = bql.Request(screen, securityid)
res = bq.execute(req)
#let's just see the first 20
data = res[0].df().head(20)
data

,ORIG_IDS,ID()
ID,,
AS779476 Corp,VOD LN Equity,AS779476 Corp
EJ552316 Corp,VOD LN Equity,EJ552316 Corp
JK203775 Corp,VOD LN Equity,JK203775 Corp
EJ372541 Corp,VOD LN Equity,EJ372541 Corp
EI109926 Corp,VOD LN Equity,EI109926 Corp
ZR549306 Corp,VOD LN Equity,ZR549306 Corp
JK189339 Corp,VOD LN Equity,JK189339 Corp
EG212194 Corp,VOD LN Equity,EG212194 Corp
AQ340827 Corp,VOD LN Equity,AQ340827 Corp


#### 3.3.3 Filter Practice

Get Bonds for 2007 HK Equity and 2777 HK Equity which are putable and maturity dates are on or after 1/1/2020?

In [17]:
# Enter Answer here
bonds = bq.univ.bonds(['2007 HK Equity','2777 HK Equity'])
filter_put = bq.data.putable() == 'True'
filter_mat = bq.data.maturity() >= '2020-01-01'
criteria = bq.func.and_(filter_put, filter_mat)
screen = bq.univ.filter(bonds, criteria)
#Define our output
securityid = bq.data.ID()

req = bql.Request(screen, securityid)
res = bq.execute(req)
data = res[0].df().head(20)
data

,ORIG_IDS,ID()
ID,,
AL294988 Corp,2007 HK Equity,AL294988 Corp
JK689218 Corp,2777 HK Equity,JK689218 Corp
AW437203 Corp,2777 HK Equity,AW437203 Corp
ZS486906 Corp,2777 HK Equity,ZS486906 Corp
ZS486935 Corp,2777 HK Equity,ZS486935 Corp
AL970947 Corp,2777 HK Equity,AL970947 Corp
QZ843395 Corp,2777 HK Equity,QZ843395 Corp
AV941107 Corp,2777 HK Equity,AV941107 Corp
QZ144205 Corp,2007 HK Equity,QZ144205 Corp


<a id='learn2-topic3.4'></a>

### 3.4 Index Members and Grouping:



- BQL enables us to retrieve the members of an index with bq.univ.members()
- Membership access is in line with your terminal permissioning

In [18]:
#Get the amount issued for each member of the Bloomberg-Barclays Pan Euro Index
universe = bq.univ.members('LP01TREU Index')
amt = bq.data.amt_issued() / 10 ** 6

req = bql.Request(universe, amt)
res = bq.execute(req)
data = res[0].df().head(20)
data

,CURRENCY_OF_ISSUE,MULTIPLIER,CURRENCY,AMT_ISSUED()/1000000
ID,,,,
AN880344 Corp,EUR,1.0,EUR,635.0
AW898691 Corp,EUR,1.0,EUR,600.0
ZR870745 Corp,EUR,1.0,EUR,600.0
AO066639 Corp,EUR,1.0,EUR,200.0
AQ787317 Corp,EUR,1.0,EUR,300.0
AQ435636 Corp,GBP,1.0,GBP,430.0
AP096320 Corp,EUR,1.0,EUR,270.0
AP105955 Corp,GBP,1.0,GBP,225.0
AP138030 Corp,EUR,1.0,EUR,500.0


In [19]:
#Retrieving membership data as of a particular point
#Universe with dates parameter
universe = bq.univ.members('LP01TREU Index', dates='2018-01-01')
#Get Amount Issued
amt = bq.data.amt_issued() / 10 ** 6

req = bql.Request(universe, amt)
res = bq.execute(req)
data = res[0].df().head(20)
data

,CURRENCY_OF_ISSUE,MULTIPLIER,CURRENCY,ID_DATE,AMT_ISSUED()/1000000
ID,,,,,
AF114752 Corp,CHF,1.0,CHF,2018-01-01,225.0
AF233280 Corp,EUR,1.0,EUR,2018-01-01,402.7
AF271439 Corp,EUR,1.0,EUR,2018-01-01,500.0
AF271440 Corp,EUR,1.0,EUR,2018-01-01,1250.0
AF271441 Corp,EUR,1.0,EUR,2018-01-01,500.0
AF280870 Corp,EUR,1.0,EUR,2018-01-01,365.0
AL029628 Corp,EUR,1.0,EUR,2018-01-01,725.0
AL030094 Corp,EUR,1.0,EUR,2018-01-01,500.0
AL044896 Corp,GBP,1.0,GBP,2018-01-01,220.0


We can add onto the query to filter only for bonds issued after a specific date

In [20]:
#Get the amount issued on bonds within the Pan-EUR HY Index issued after 2018

#Define our universe and our filter criteria
universe = bq.univ.members('LP01TREU Index')
criteria = bq.data.issue_dt() > '2017-12-31'
filtered_univ = bq.univ.filter(universe, criteria)
#Define our fields we wish to see
amt = bq.data.amt_issued() / 10 ** 6
issue = bq.data.issue_dt()
#Create an Ordered Dictionary of our fields
flds = OrderedDict()
flds['Amount_Issued'] = amt
flds['Issue_Date'] = issue

req = bql.Request(filtered_univ, flds)
res = bq.execute(req)
#We use Pandas here to align the data: combined df only works when we have the same metadata columns in the data we wish to retrieve
tbl = pd.DataFrame({r.name:r.df()[r.name] for r in res})
tbl.head(20)

,Amount_Issued,Issue_Date
ID,,
AW898691 Corp,600.0,2019-02-04
ZR870745 Corp,600.0,2019-10-15
AQ787317 Corp,300.0,2018-01-31
AQ435636 Corp,430.0,2018-01-24
ZO315830 Corp,300.0,2020-09-10
ZO366735 Corp,500.0,2020-09-14
ZP513192 Corp,400.0,2020-01-22
AZ851869 Corp,720.0,2019-08-13
ZO908055 Corp,525.0,2020-10-21


<a id='learn2-topic3.5'></a>

### 3.5 Grouping:



BQL allows us to perform aggregated calculations, as well as group by custom criteria.
The key is the function bq.func.group(), which allows BQL to understand the data are connected in one bucket.

In [21]:
#Get the total amount outstanding in USD for all members of the Bloomberg Pan-EUR HY Index
#Define our universe of index members
universe = bq.univ.members('LP01TREU Index')
#group our data and perform a statistical function
total_amt = bq.data.amt_issued(CURRENCY='USD').group().sum() / 10 ** 6

req = bql.Request(universe, {'Total Amount Issued':total_amt})
res = bq.execute(req)

data = res[0].df()
data

,CURRENCY_OF_ISSUE,MULTIPLIER,CURRENCY,ORIG_IDS,Total Amount Issued
ID,,,,,
IdentityGroup,None,1.0,USD,None,516095.179517


#### 3.5.1 Grouping Practice I

How many loans are currently active for Tesla's Credit Family?

In [22]:
#Enter answer below
loans = bq.univ.loans('TSLA US Equity', issuedby='Credit_family')
securityid = bq.data.ID()
total_id = securityid.group().count()

req = bql.Request(loans, total_id)
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS:0,ORIG_IDS:1,COUNT(GROUP(ID()))
ID,,,
IdentityGroup,BL174800 Corp,TSLA US Equity,1


In [23]:
#Combine with our filter to get aggregated data for our screened universe

universe = bq.univ.members('LP01TREU Index')
criteria = bq.data.issue_dt() > '2017-12-31'
filtered_univ = bq.univ.filter(universe, criteria)

total_amt = bq.data.amt_outstanding(CURRENCY='USD').group().sum()/10**6

req = bql.Request(filtered_univ, {'Total Amount Issued':total_amt})
res = bq.execute(req)

data = res[0].df()
data

,CURRENCY_OF_ISSUE,MULTIPLIER,CURRENCY,ORIG_IDS,Total Amount Issued
ID,,,,,
IdentityGroup,None,1.0,USD,None,349385.818439


#### 3.5.2 Grouping by criteria



<img src="../../Visualisations/BQL Grouping.jpg" style="width: 600px;"/>

Using group() we are also able to group and bucket data by category, such as country, rating or industry.

In [24]:
#Getting the average amount issued in USD for the bonds in the Bloomberg Barclays Pan Euro which were issued after 2017.

#Let's define our universe first
universe = bq.univ.members('LP01TREU Index')
criteria = bq.data.issue_dt() > '2017-12-31'
filtered_univ = bq.univ.filter(universe, criteria)
#Next we want to define our data we wish to group by and retrieve
amt_issue = bq.data.amt_issued(CURRENCY='USD')
sector = bq.data.classification_name('BCLASS','3')
sector_avg_amt = amt_issue.group(sector).avg() / 10 ** 6

req = bql.Request(filtered_univ, {'Avg Amt Issued':sector_avg_amt})
res = bq.execute(req)

data = res[0].df()
data

,CURRENCY_OF_ISSUE,MULTIPLIER,CURRENCY,ORIG_IDS,"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.BCLASS,CLASSIFICATION_LEVEL.3)",Avg Amt Issued
ID,,,,,,
Banking,None,1.0,USD,None,Banking,563.749176
Basic Industry,EUR,1.0,USD,None,Basic Industry,555.457111
Brokerage Assetmanagers Exchanges,EUR,1.0,USD,None,Brokerage Assetmanagers Exchanges,559.605500
Capital Goods,None,1.0,USD,None,Capital Goods,585.034663
Communications,None,1.0,USD,None,Communications,831.365148
Consumer Cyclical,None,1.0,USD,None,Consumer Cyclical,623.846585
Consumer Non-Cyclical,None,1.0,USD,None,Consumer Non-Cyclical,644.384405
Electric,None,1.0,USD,None,Electric,631.146692
Energy,EUR,1.0,USD,None,Energy,657.099759


#### 3.5.3 Grouping Practice II

What is the total amount outstanding in SGD of bonds by S&P credit rating from the Barclays US Corporate HY Index (LF98TRUU) which mature after 2028?

In [ ]:
#Enter answer here
#Screen index
index = bq.univ.members('LF98TRUU Index')
criteria_mat = bq.data.maturity() >= '2028-12-31'
filtered_index = bq.univ.filter(index, criteria_mat)
#Retrieve amt out
amt_out = bq.data.amt_outstanding(CURRENCY='SGD')
rating = bq.data.credit_rating(CREDIT_RATING_SOURCE='SANDP')
amt_out_by_rating = amt_out.group(rating).sum() / 10 ** 6

req = bql.Request(filtered_index, {'Amt Out by Rating':amt_out_by_rating})
res = bq.execute(req)
data = res[0].df()
data

#### 3.5.4 Weighted Average Grouping



- Also grouping, but with two parameters

In [ ]:
#Weighted Average spread by sector of my LBUSTRUU

#My Index
index = bq.univ.members('LBUSTRUU Index')
#clean for NaN
sprd = bq.data.spread(SPREAD_TYPE='Z')
criteria = sprd > 0
cleaned_index = bq.univ.filter(index, criteria)
#Weighted Average Spread
weights = bq.data.ID()['WEIGHTS']
sector = bq.data.classification_name()
weighted_avg = sprd.group(sector).wavg(weights.group(sector))

req = bql.Request(cleaned_index, {'Weighted Avg Spread by Sector':weighted_avg})
res = bq.execute(req)

data = res[0].df()
data

<a id='learn2-topic3.6'></a>
## 3.6 Portfolio Access



- Use the members function to also access the members of your portfolio using the portfolio ID.
- the syntax is members('portfolio ID',type=PORT)

<img src="../../Visualisations/Portfolio.jpg" style="width: 600px;"/>

In [ ]:
#Select the members of your portfolio in PRTU
portfolio = bq.univ.members('U5594175-58', type='PORT')
securityid = bq.data.ID()

req = bql.Request(portfolio, securityid)
res = bq.execute(req)
data = res[0].df()
data

In [ ]:
#If we want to see the weights, this is one of the data associated with ID

portfolio = bq.univ.members('U5594175-58', type='PORT')
#Of the output, select the columns titles 'WEIGHTS'
weights = bq.data.ID()['WEIGHTS']

req = bql.Request(portfolio, weights)
res = bq.execute(req)

data = res[0].df()
data

In [ ]:
#Weighted Average spread by sector of my portfolio

#My portfolio
portfolio = bq.univ.members('U5594175-58', type='PORT')
#Weighted Average Spread
sprd = bq.data.spread(SPREAD_TYPE='Z')
weights = bq.data.ID()['WEIGHTS']
sector = bq.data.classification_name()
weighted_avg = sprd.group(sector).wavg(weights.group(sector))

req = bql.Request(portfolio,{'Weighted Avg Spread by Sector':weighted_avg})
res = bq.execute(req)

data = res[0].df()
data

#### 3.6.1 Grouping Practice III

What is the breakdown of your portfolio, weighted by country of risk?

In [ ]:
#Enter answer here
#My portfolio
portfolio = bq.univ.members('U5594175-58', type='PORT')
#Weighted Average Yield
yld = bq.data.yield_()
weights = bq.data.id()['WEIGHTS']
country = bq.data.cntry_of_risk()
weighted_avg = yld.group(country).wavg(weights.group(country))

req = bql.Request(portfolio,{'Weighted Avg Yield by Country':weighted_avg})
res = bq.execute(req)
data = res[0].df()
data

<a id='learn2-topic3.7'></a>
### 3.7 Universes:



BQL can access a list of securities, portfolio, or the entire fixed income universe on Bloomberg.

- The key here is bq.univ.bondsUniv(), loansUniv() or debtUniv() - the latter incorporates bonds, loans and preferreds.
- Possible options are 'active', 'all' and 'matured'

<img src="../../Visualisations/SRCH.jpg" style="width: 600px;"/>

In [ ]:
#Find the average OAS spread for all active bonds in the financial sector which have a fixed coupon, expiring less than 3 months from now, and a rating greater than AA- 

#Filter the universe of bonds to only include bonds which fit the above criteria
univ = bq.univ.bondsuniv('Active')
#Define our screening criteria
filter_cpn = bq.data.cpn_typ() == 'FIXED'
filter_sector = bq.data.classification_name('BICS','1') == 'Financials'
filter_mat = bq.data.maturity() < '3M'
filter_rat = bq.data.bb_composite() >= 'AA-'
#Combine our criteria using the and_ function, which can takes two inputs
criteria = bq.func.and_(bq.func.and_(bq.func.and_(filter_cpn,filter_sector), filter_mat), filter_rat)
filtered_univ = bq.univ.filter(univ, criteria)
#Group the filtered universe by spread in order to get the average spread
sprd = bq.data.SPREAD(SPREAD_TYPE='OAS')
avg_sprd = sprd.group().avg()

req = bql.Request(filtered_univ, {'Avg OAS Spread':avg_sprd},with_params={'mode':'cached'})
res = bq.execute(req)

data = res[0].df()
data

#### 3.7.1 Universe Practice

How many active USD term loans are outstanding per sector (using BICS, Level 1)?

In [ ]:
#Enter answer here
loans = bq.univ.loansuniv('Active')
criteria_crncy = bq.data.crncy() == 'USD'
criteria_term = bq.data.loan_typ() == 'Term'
crtieria = bq.func.and_(criteria_crncy, criteria_term)
filtered_loans = bq.univ.filter(loans, criteria)
#count ID by sector
securityid = bq.data.id()
sector = bq.data.classification_name('BICS','1')
loans_by_sector = securityid.group(sector).count()

req = bql.Request(loans,{'USD Term Loans by Sector':loans_by_sector})
res = bq.execute(req)
data = res[0].df()
data

[&uarr; Return to Top](#learn2-top)

<a id='learn2-topic4'></a>

<span style="color:green; font-size:2em"> 4. BQL Basics Summary </span>



### BQL

- Simple and efficient data retrieval
- Screen bonds/loans by issuer, index members, portfolio or whole universe
- Aggregate data on index members and filtered bonds

### Benefits

- Save time and effort cleaning, manipulating, sorting data
- Data-efficient analysis
- Focus on generating ideas

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="1.1. FI Basics - Workbook.ipynb">&larr; Back to the Part I Workbook</a>&emsp;&emsp;
    <a href="#learn2-top">&uarr; Back to Top </a>&emsp;&emsp;
    <a href="1.3. FI Advanced - Workbook.ipynb">Next Topic: FI Advanced Workbook &rarr;</a>
    <br>

</p>